# Import Libraries

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from scipy.signal import find_peaks
from height_experiments.tools import extract_exercises, smooth_column
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
from height_experiments.tools import compute_height_initial_velocity

# Connect to Firebase

In [ ]:
cred = credentials.Certificate('firebase_python.json')

In [ ]:
firebase_admin.initialize_app(cred)

In [ ]:
db = firestore.client()

In [ ]:
collection = db.collection('signals_db')

In [ ]:
res = collection.get() # returns a list
res = [i.to_dict() for i in res]
document = sorted(res, key=lambda x: x["info"]["date"].timestamp(), reverse=True)[0]["info"]

In [ ]:
document["date"]

# Adding time values

In [ ]:
for index in range(len(document["timeStampSamples"])):
    document["sensorAcc"][index]["time"] = document["timeStampSamples"][index]

# Smooth signal and extract exercises

In [ ]:
df = pd.DataFrame(document["sensorAcc"])
df = smooth_column(df, "valueZ")
df = df.iloc[::-1].reset_index(drop=True)

peaks, _ = find_peaks(df['valueZ_Smooth'])
exercises = extract_exercises(df.to_dict(orient='records'), peaks.tolist(), 'valueZ_Smooth')

df.head()

# Show signals

In [ ]:
fig = px.line(df, y='valueZ_Smooth', title='Linear Acceleration Z over Time')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['valueZ_Smooth'], mode='lines', name='Time Series'))
fig.add_trace(go.Scatter(x=df.index[peaks], y=df['valueZ_Smooth'][peaks], mode='markers', marker=dict(size=10, color='red'), name='Peaks'))
fig.update_layout(title='Time Series Data with Peaks', xaxis_title='Time', yaxis_title='Value')
fig.show()

# Show last repetitions

In [ ]:
for num, exercise in enumerate(exercises):
    df_ex = pd.DataFrame(exercise)
    jump_height = compute_height_initial_velocity(df_ex, "valueZ_Smooth")
    fig = px.line(df_ex, y='valueZ_Smooth', title=f'Jump #{num}: {jump_height}m')
    fig.show()

# Comparing App Results

In [ ]:
jumps_flutter = document["jumpHeights"]
for num, exercise in enumerate(exercises):
    df_ex = pd.DataFrame(exercise)
    jump_height = compute_height_initial_velocity(df_ex, "valueZ_Smooth")
    jump_app = jumps_flutter[num]
    print(f"Jump #{num}: Python: {jump_height}m ; Flutter: {jump_app}")